In [1]:
from glob import glob

import torch
from scene import Scene
import os
import open3d as o3d
#from open3d import JVisualizer
import sys
import numpy as np
from tqdm import tqdm
from os import makedirs
from gaussian_renderer import render
import torchvision
from argparse import ArgumentParser, Namespace

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.offline import init_notebook_mode, iplot

from utils.general_utils import safe_state
from arguments import ModelParams, PipelineParams, get_combined_args
from gaussian_renderer import GaussianModel

from scene.colmap_loader import (
    read_extrinsics_text, read_intrinsics_text, qvec2rotmat,
    read_extrinsics_binary, read_intrinsics_binary,
    read_points3D_binary, read_points3D_text
)
from scene.dataset_readers import (
    SceneInfo, readColmapCameras
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
SOURCE_DATA_PATH_LIST = [
    "/home/hjp/KUAICV/NIPS2024_LOCAL/datasets/tandt/train/"
]

# read camera parameter files and pointcloud files

SCENE_IDX = 0

SOURCE_DATA_PATH = SOURCE_DATA_PATH_LIST[SCENE_IDX]

cameras_extrinsic_file = os.path.join(SOURCE_DATA_PATH, "sparse/0", "images.bin")
cameras_intrinsic_file = os.path.join(SOURCE_DATA_PATH, "sparse/0", "cameras.bin")
pointcloud_bin_file = os.path.join(SOURCE_DATA_PATH, "sparse/0", "points3D.bin")
pointcloud_ply_file = os.path.join(SOURCE_DATA_PATH, "sparse/0", "points3D.ply")

cam_extrinsics = read_extrinsics_binary(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_binary(cameras_intrinsic_file)
xyz, rgb, _ = read_points3D_binary(pointcloud_bin_file)

cam_info_list = readColmapCameras(
    cam_extrinsics  = cam_extrinsics,
    cam_intrinsics  = cam_intrinsics,
    images_folder   = os.path.join(SOURCE_DATA_PATH, "images")
)

pcd = o3d.io.read_point_cloud(pointcloud_ply_file)
# get numpber of points
print(len(pcd.points), "points")

In [ ]:
from plyfile import PlyData, PlyElement


MODEL_PATH = "/home/hjp/KUAICV/NIPS2024_LOCAL/gaussian-splatting/output/e991fed0-9/"
out_pcd_path = os.path.join(
    MODEL_PATH,
    "point_cloud/iteration_30000/point_clout.ply"
)
out_pcd_path = "/home/hjp/KUAICV/NIPS2024_LOCAL/gaussian-splatting/output/e991fed0-9/point_cloud/iteration_30000/point_clout.ply"
out_pcd_path = "/home/hjp/KUAICV/NIPS2024_LOCAL/gaussian-splatting/output/e991fed0-9/point_cloud/iteration_30000/point_cloud.ply"

ply_data = PlyData.read(out_pcd_path)

xyz = np.stack((
    np.asarray(ply_data.elements[0]["x"]),
    np.asarray(ply_data.elements[0]["y"]),
    np.asarray(ply_data.elements[0]["z"])),
    axis=1
)

In [ ]:
xyz.shape